In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import gdown
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import micro
import gtsa

## Download sample data

In [ ]:
url = 'https://drive.google.com/drive/folders/1CxLSaQl8VzsGrKUJUVjRUh2yYwWJGNrl'
dem_folder='../data/dems'
# gdown.download_folder(url, output=dem_folder, quiet=False, use_cookies=False)

url = 'https://drive.google.com/drive/folders/1KVlr6hueLEqRIfizw74y0VAR_m9i6jlI'
vector_folder='../data/vectors'
# gdown.download_folder(url, output=vector_folder, quiet=False, use_cookies=False)

In [ ]:
!tree ../data/

In [ ]:
dsm_files = sorted(Path(dem_folder).glob('ws*.tif'))
dsm_files = [i.as_posix() for i in dsm_files] # convert from posix path to string

In [ ]:
dsm_files

## Select analysis ready data
- Assuming SfM and lidar collected on the same data are only used for validation of the SfM data
- Assuming the lidar is always "better" and will thus be used for the quantitative measurements and time series analysis

In [ ]:
dsm_files = ['../data/dems/wsyr1_20201109_KID13_LiDAR.tif',
             # '../data/dems/wsyr1_20201109_KID13_sfm.tif',
             '../data/dems/wsyr1_20210208_KID13_sfm.tif',
             '../data/dems/wsyr1_20210406_KID13_sfm.tif',
             '../data/dems/wsyr1_20210513_KID13_LiDAR.tif',
             # '../data/dems/wsyr1_20210513_KID13_sfm.tif',
            ]

## Parse time stamps

In [ ]:
strings = gtsa.io.parse_timestamps(dsm_files, date_string_pattern='_........_')
date_strings = [x[1:-1] for x in strings]

# ensure chronological sorting
date_strings, dsm_files = list(zip(*sorted(zip(date_strings, dsm_files))))
date_times = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings]

In [ ]:
list(zip(date_strings, dsm_files))

## Create stack
- resample to reference DEM grid and save as netcdf

In [ ]:
reference_dem = dsm_files[0]

ds = gtsa.io.xr_stack_geotifs(
    dsm_files,
    date_times,
    reference_dem,
    resampling='cubic',
    save_to_nc=True,
    # nc_out_dir=None,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

## Plot first time stamp

In [ ]:
ds['band1'].isel(time=0).plot(aspect='equal', 
                              size=5)

## Compute differences for stack

In [ ]:
ds_diff = ds['band1'].diff(dim='time')

## Plot first difference map

In [ ]:
ds_diff.isel(time=0).plot(cmap='RdBu',
                          vmin=-0.1,
                          vmax=0.1,
                          aspect='equal',
                          size=5)

## Read shapefile

In [ ]:
shapefiles = [x.as_posix() for x in sorted(Path(vector_folder).glob('*.shp'))]
gdf = gpd.read_file(shapefiles[0])

## Plot shapefile

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

ds_diff.isel(time=0).plot(ax=ax,
                          cmap='RdBu',
                          vmin=-0.1,
                          vmax=0.1,
                          )

xlim = ax.get_xlim()
ylim = ax.get_ylim()

gdf.plot(ax=ax, facecolor='none', edgecolor='black')

## Clip stack for each geometry

In [ ]:
geometry_0 = gpd.GeoDataFrame(gpd.GeoSeries(gdf.iloc[0]['geometry']), columns=['geometry'])
geometry_1 = gpd.GeoDataFrame(gpd.GeoSeries(gdf.iloc[1]['geometry']), columns=['geometry'])

In [ ]:
ds_diff_clip_geom_0 = ds_diff.rio.clip(geometry_0['geometry'])
ds_diff_clip_geom_1 = ds_diff.rio.clip(geometry_1['geometry'])

## Plot first time step in clipped stack

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

ds_diff_clip_geom_0.isel(time=0).plot(ax=ax,
                                      cmap='RdBu',
                                      vmin=-0.1,
                                      vmax=0.1,
                                      add_colorbar=False,
                                     )

geometry_0.plot(ax=ax, facecolor='none', edgecolor='green')

ds_diff_clip_geom_1.isel(time=0).plot(ax=ax,
                                      cmap='RdBu',
                                      vmin=-0.1,
                                      vmax=0.1,
                                      cbar_kwargs={'extend': 'both'},
                                     )

geometry_1.plot(ax=ax, facecolor='none', edgecolor='purple')

ax.set_xlim(xlim)
ax.set_ylim(ylim);

## Plot time series for mean difference value

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ds_diff_clip_geom_0.mean(dim=['x','y']).plot.scatter(ax=ax,color='green')
ds_diff_clip_geom_1.mean(dim=['x','y']).plot.scatter(ax=ax,color='purple')